In [1]:
import pandas as pd
import numpy as np
import os
from autocorrect import Speller

In [2]:
directory = 'Interpromo2020/Interpromo2020/'

files = os.listdir(directory+'All Data/')
files_xlsx = [directory+'All Data/'+f for f in files if os.path.isfile(directory+'All Data/'+f) and f.endswith('.xlsx')]
files_csv  = [directory+'All Data/'+f for f in files if os.path.isfile(directory+'All Data/'+f) and f.endswith('.csv')]

files = os.listdir(directory)
files_xlsx2 = [directory+f for f in files if os.path.isfile(directory+f) and f.endswith('.xlsx')]
files_xlsx.extend(files_xlsx2)

In [3]:
files_xlsx

['Interpromo2020/Interpromo2020/All Data/KAYAK_AIRLINE_RATE_GLOBAL.xlsx',
 'Interpromo2020/Interpromo2020/All Data/ALL_DATA.xlsx',
 'Interpromo2020/Interpromo2020/All Data/AIRCRAFT_DETAIL_V2.xlsx',
 'Interpromo2020/Interpromo2020/All Data/DATA_KAYAK_ROUTE.xlsx',
 'Interpromo2020/Interpromo2020/All Data/ALL_DATA_ANNOTATE.xlsx',
 'Interpromo2020/Interpromo2020/All Data/OTHERS_DATA_ANNOTATE.xlsx',
 'Interpromo2020/Interpromo2020/TWITTER.xlsx']

In [4]:
df = pd.read_excel(files_xlsx[-1])
df.head()

,contains_photo,contains_video,created_at,entities,favorite_count,is_quote_status,lang,quoted_status,quoted_status_id_str,retweet_count,text,user
0,1,0,2019-04-02 14:38:26,"{'hashtags': ['airBaltic'], 'symbols': [], 'us...",72,0,en,NaN,NaN,13,Good news for passengers on the Prague route! ...,"{'id_str': '26726695', 'name': 'airBaltic', 's..."
1,1,0,2019-04-01 19:00:01,"{'hashtags': ['Airbus', 'A220', 'avgeek', 'Afr...",111,0,en,NaN,NaN,19,"Thanks to the GTF, Mount Kilimanjaro 🗻 was bre...","{'id_str': '20814874', 'name': 'Pratt & Whitne..."
2,1,0,2019-04-01 10:40:21,"{'hashtags': ['A220'], 'symbols': [], 'user_me...",29,0,en,NaN,NaN,6,"Sunbathing in Bucharest: YL-CSM, @Airbus #A220...","{'id_str': '1403040254', 'name': 'Aeronews', '..."
3,1,0,2019-03-31 10:20:23,"{'hashtags': [], 'symbols': [], 'user_mentions...",74,0,en,NaN,NaN,9,Today we opened long-awaited @airBaltic route ...,"{'id_str': '52174696', 'name': 'Martin Gauss',..."
4,1,0,2019-03-31 06:10:27,"{'hashtags': ['airBaltic'], 'symbols': [], 'us...",58,0,en,NaN,NaN,9,The new #airBaltic route between Riga and Dubl...,"{'id_str': '26726695', 'name': 'airBaltic', 's..."


### Slot class

In [5]:
def get_slot_date(text): return '08/01/2020' if 'date' in  text else np.nan
def get_slot_city_dep(text): return 'Paris' if np.random.rand() > 0.9 else np.nan
def get_slot_city_arr(text): return 'London' if np.random.rand() > 0.9 else np.nan
def get_slot_airport_dep(text): return 'CDG' if np.random.rand() > 0.9 else np.nan
def get_slot_airport_arr(text): return 'LCY' if np.random.rand() > 0.9 else np.nan
def get_slot_flight_number(text): return 'BA2490' if np.random.rand() > 0.9 else np.nan

In [6]:
class Slot:
    def __init__(self, question, detection_function):
        self.question = question
        self.fun = detection_function
    
    def get_value(self, text):
        return self.fun(text)

In [7]:
Slot_date = Slot('What is the date of the flight?', get_slot_date)
Slot_city_dep = Slot('What is the city of departure?', get_slot_city_dep)
Slot_city_arr = Slot('What is the city of arrival?', get_slot_city_arr)
Slot_airport_dep = Slot('What is the airport of departure?', get_slot_airport_dep)
Slot_airport_dep = Slot('What is the airport of arrival?', get_slot_airport_arr)
Slot_flight = Slot('What is the flight number?', get_slot_flight_number)

### Intent class

In [8]:
class Intent:
    def __init__(self, name, slots):
        self.name = name
        self.slots = slots

In [9]:
Intent_com = Intent('Communication',[])
Intent_flight = Intent('Fligh',[Slot_date, Slot_city_dep, Slot_city_arr, 
                                Slot_airport_dep, Slot_airport_dep, Slot_flight])
Intent_path = Intent('Path',[Slot_date, Slot_city_dep, Slot_city_arr, 
                                Slot_airport_dep, Slot_airport_dep, Slot_flight])

In [10]:
dict_intents = {0 : Intent_com,
                1 : Intent_path,
                2 : Intent_flight}

### Interest filter

In [11]:
# return if a text is intersting for us (filter)
def isOfInterest(text):
    return np.random.rand() > 0.99

df_textOfInterest = df.loc[:,['text']][df['text'].apply(isOfInterest) == True]
df_textOfInterest.head()

,text
226,Order of 60 A220-300s to replace our A318 and ...
298,Happy to fly directly from Riga to #Stuttgart ...
663,A6-SHJ | Airbus A320-232(CJ)(WL) Prestige | Ru...
703,An Alaska Airlines A320 landing in Las Vegas. ...
836,[4K VIDEO] 2️⃣ #ALASKAAIRLINES #AIRBUS #A320-2...


### Intent detection

In [12]:
# return the intend corresponding to the text
def detect_intent(model, text):
    nb_class = 3
    return int(np.random.rand()*3) % nb_class

In [13]:
dict_intents[detect_intent('','')].name

'Path'

# Intent communication

In [14]:
def get_context(text):
    context = np.random.rand(100)
    sentiment = int(np.random.rand()*3) % 3 - 1
    sentiment = np.nan if np.random.rand() > 0.9 else sentiment
    return context, sentiment

def get_context_words(context):
    return ['window','plane']

def make_near_question(words_list):
    return (words_list[0] + ' is your concern. What do you recommand to improve our service?')

def make_sentence_ok(context, sentiment):
    return 'ok thanks for your reply, we can understand that.'

def make_question_open(context, sentiment):
    return 'What can you tell us more about your experience?'

def make_question_close(context, sentiment):
    context_words = get_context_words(context)
    return make_near_question(context_words)

##### Test

In [15]:
# autocorrection:
spell = Speller(lang='en')

In [16]:
current_text = df_textOfInterest['text'].tolist()[0]
current_text_corrected = spell(current_text)

print(current_text)
print(current_text_corrected)

Order of 60 A220-300s to replace our A318 and A319, generating 20% less CO2 emissions than comparable aircraft and twice as quiet ✈️                                                                         
More information 👉 https://t.co/wEXLQBGqy2 https://t.co/qOlqYWwqD1
Order of 60 A220-300s to replace our A318 and A319, generating 20% less CO2 emissions than comparable aircraft and twice as quiet ✈️                                                                         
More information 👉 http://t.co/wEXLQBGqy2 http://t.co/qOlqYWwqD1


In [17]:
detected_intent = detect_intent('',current_text)
print("detected intent:", detected_intent, ":", dict_intents[detected_intent].name)

detected intent: 2 : Fligh


In [18]:
context, sentiment = get_context(current_text)

sent_ok = make_sentence_ok(context, sentiment)
sent_open = make_question_open(context, sentiment)
sent_close = make_question_close(context, sentiment)

print(sent_ok)
print(sent_open)
print(sent_close)

ok thanks for your reply, we can understand that.
What can you tell us more about your experience?
window is your concern. What do you recommand to improve our service?


# NLTK

In [19]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

In [20]:
current_text

'Order of 60 A220-300s to replace our A318 and A319, generating 20% less CO2 emissions than comparable aircraft and twice as quiet ✈️                                                                         \nMore information 👉 https://t.co/wEXLQBGqy2 https://t.co/qOlqYWwqD1'

In [21]:
sent = nltk.word_tokenize(current_text)
post = nltk.pos_tag(sent)

In [22]:
post

[('Order', 'NN'),
 ('of', 'IN'),
 ('60', 'CD'),
 ('A220-300s', 'JJ'),
 ('to', 'TO'),
 ('replace', 'VB'),
 ('our', 'PRP$'),
 ('A318', 'NNP'),
 ('and', 'CC'),
 ('A319', 'NNP'),
 (',', ','),
 ('generating', 'VBG'),
 ('20', 'CD'),
 ('%', 'NN'),
 ('less', 'JJR'),
 ('CO2', 'NNP'),
 ('emissions', 'NNS'),
 ('than', 'IN'),
 ('comparable', 'JJ'),
 ('aircraft', 'NN'),
 ('and', 'CC'),
 ('twice', 'RB'),
 ('as', 'IN'),
 ('quiet', 'JJ'),
 ('✈️', 'NNP'),
 ('More', 'JJR'),
 ('information', 'NN'),
 ('👉', 'NN'),
 ('https', 'NN'),
 (':', ':'),
 ('//t.co/wEXLQBGqy2', 'JJ'),
 ('https', 'NN'),
 (':', ':'),
 ('//t.co/qOlqYWwqD1', 'NN')]

In [23]:
import spacy
nlp = spacy.load('en_core_web_sm')